In [13]:
import numpy as np

def _union(lists):
    union_set = set()

    for l in lists:
        union_set = union_set.union(set(l))
    return union_set

def _is_conservative(elements, lists):
    for e in elements:
        conservative = False

        for l in lists:
            if e not in l:
                conservative = True
                break
        if not conservative:
            return False
    return True

def _is_covering(elements, lists):
    return set(elements) == _union(lists)

def _pick_targets(nb_nodes, min_nb_target, max_nb_target, nb_interventions, nb_max_iteration=100000, cover=False, conservative=True):
    nodes = np.arange(nb_nodes)
    not_correct = True
    i = 0

    if(max_nb_target == 1):
        intervention = np.sort(np.random.choice(nb_nodes, nb_interventions, replace=False))
        targets = [[i] for i in intervention]

    else:
        while(not_correct and i < nb_max_iteration):
            targets = []
            not_correct = False
            i += 1

            # pick targets randomly (without repeat)
            j = 0
            while j < nb_interventions:
                nb_targets = np.random.randint(min_nb_target, max_nb_target+1, 1)
                intervention = np.sort(np.random.choice(nb_nodes, nb_targets, replace=False))
                if intervention not in targets:
                    targets.append(intervention)
                    j += 1

            # apply rejection sampling
            if cover and not _is_covering(nodes, targets):
                not_correct = True
            if conservative and not _is_conservative(nodes, targets):
                not_correct = True

        if i == nb_max_iteration:
            raise ValueError("Could generate appropriate targets. \
                                 Exceeded the maximal number of iterations")

        for i, t in enumerate(targets):
            targets[i] = np.sort(t)

    return targets

In [14]:
import warnings
import pandas as pd
import networkx as nx
from dynotears_data_generators.wrappers import gen_stationary_dyn_net_and_df, generate_dataframe_dynamic
def gen_df_from_g(
        g, p, intra_nodes, inter_nodes,
        num_nodes, 
        n_samples = 100, 
        max_data_gen_trials = 10,
        sem_type: str = "linear-gauss",
        noise_scale: float = 1,
        ): # generate time sery data
    with np.errstate(over="raise", invalid="raise"):
        burn_in = max(n_samples // 10, 50)

        simulate_flag = True

        while simulate_flag:
            max_data_gen_trials -= 1
            if max_data_gen_trials <= 0:
                simulate_flag = False

            try:
                simulate_graphs_flag = False

                # generate single time series
                df = (
                    generate_dataframe_dynamic(
                        g,
                        n_samples=n_samples + burn_in,
                        sem_type=sem_type,
                        noise_scale=noise_scale,
                    )
                    .loc[burn_in:, intra_nodes + inter_nodes]
                    .reset_index(drop=True)
                )

                if df.isna().any(axis=None):
                    continue
            except (OverflowError, FloatingPointError):
                continue
            if (df.abs().max().max() < 1e3) or (max_data_gen_trials <= 0):
                simulate_flag = False
        if max_data_gen_trials <= 0:
            warnings.warn(
                "Could not simulate data, returning constant dataframe", UserWarning
            )

            df = pd.DataFrame(
                np.ones((n_samples, num_nodes * (1 + p))),
                columns=intra_nodes + inter_nodes,
            )
    return df

In [15]:
import csv
import os
import pandas as pd
from cdt.metrics import get_CPDAG

def save_dag_cpdag(fname_radical, i_dataset, adjacency_matrix):
    dag_path = os.path.join(fname_radical, f'DAG{i_dataset}.npy')
    cpdag_path = os.path.join(fname_radical, f'CPDAG{i_dataset}.npy')
    cpdag = get_CPDAG(adjacency_matrix)

    np.save(dag_path, adjacency_matrix)
    np.save(cpdag_path, cpdag)

def save_data(fname_radical, i_dataset, data):
        data_path = os.path.join(fname_radical, f'data{i_dataset}.npy')
        np.save(data_path, data)

def to_npy(fname_radical, i_dataset, adjacency_matrix, data):
    """
    Save the generated data to the npy format,
    in two separate files: data and the adjacency matrix of the
    corresponding graph.

    Args:
        fname_radical (str): radical of the file names.
        i_dataset (int): i-th dataset
    """
    if not os.path.exists(fname_radical):
        os.makedirs(fname_radical)
    if data is not None:
        save_dag_cpdag(fname_radical, i_dataset, adjacency_matrix)
        save_data(fname_radical, i_dataset, data)
    else:
        raise ValueError("Graph has not yet been generated. \
                            Use self.generate() to do so.")
    
def _save_data(folder, i, data, regimes=None, mask=None):
    if mask is None:
        data_path = os.path.join(folder, f'data{i+1}.npy')
        np.save(data_path, data)
    else:
        data_path = os.path.join(folder, f'data_interv{i+1}.npy')
        np.save(data_path, data)

        data_path = os.path.join(folder, f'intervention{i+1}.csv')
        with open(data_path, 'w', newline="") as f:
            writer = csv.writer(f)
            writer.writerows(mask)

    if regimes is not None:
        regime_path = os.path.join(folder, f'regime{i+1}.csv')
        with open(regime_path, 'w', newline="") as f:
            writer = csv.writer(f)
            for regime in regimes:
                writer.writerow([regime])

In [17]:
import copy
from causalnex.structure.structuremodel import StructureModel
import pandas as pd
from diff_intervene_generation.causal_mechanisms import UniformCause
from dynotears_data_generators.wrappers import generate_dataframe_dynamic


def simulate_timedata(
    i_dataset = 1,
    file_path = '/home/lpw/dcdi-master/data_timesery/imperfect/',
    nb_interventions = 5,
    min_nb_target=1, 
    max_nb_target=3,
    nb_all_sample = 10,
    num_nodes = 5,
    nb_time_points = 100,
    lag = 3,
    interv_type = 'imperfect' # perfect
    ):

    # generate mask_intervention列表包含与每个点相关联的标记，regimes列表包含每个点所属的intervention编号, 共points_per_interv[j]*nb_time_points点
    div = nb_interventions + 1
    # one-liner taken from https://stackoverflow.com/questions/20348717/algo-for-dividing-a-number-into-almost-equal-whole-numbers/20348992
    points_per_interv = [nb_all_sample // div + (1 if x < nb_all_sample % div else 0)  for x in range (div)]
    # points_per_interv = int(self.nb_points / self.nb_interventions)
    print('points_per_interv:', points_per_interv)
    
    # generate true data
    g, df, intra_nodes, inter_nodes = gen_stationary_dyn_net_and_df(
        num_nodes = num_nodes, n_samples = nb_time_points*points_per_interv[0],
        p = lag, degree_intra = 3, degree_inter = 3,
        graph_type_intra = "erdos-renyi", graph_type_inter = "erdos-renyi",
        # w_min_intra = 0.5, w_max_intra = 2.0,
        # w_min_inter = 0.3, w_max_inter = 0.5,
        w_min_intra = 0.25, w_max_intra = 1.0,
        w_min_inter = 0.25, w_max_inter = 1.0,
        w_decay = 2.0,
        sem_type = "linear-gauss", noise_scale  = 1,
        max_data_gen_trials  = 3000
        )
    # w_mat = nx.to_numpy_array(g, nodelist=intra_nodes)
    # a_mat = nx.to_numpy_array(g, nodelist=intra_nodes + inter_nodes)[len(intra_nodes) :, : len(intra_nodes)]
    adj = nx.to_numpy_array(g, nodelist=intra_nodes + inter_nodes)
    # X=XW+YA+Z 1*5=1*5 5*5 + 1*5p 5p* 5 + 1*5, adj = [w_mat 0; a_mat 0], adj[5:,:5]==a_mat
    to_npy(file_path, i_dataset, np.where(adj==0,adj,1), df) # save CPDAG, DAG, data

    target_list = _pick_targets(nb_nodes=len(intra_nodes), 
                                min_nb_target=min_nb_target, max_nb_target=max_nb_target, 
                                nb_interventions=nb_interventions)
    print("target_list:", target_list)

    mask_intervention = []
    regimes = []
    mask_intervention.extend([[] for i in range(points_per_interv[0]*nb_time_points)])
    regimes.extend([0 for i in range(points_per_interv[0]*nb_time_points)])
    df_interv = copy.deepcopy(df) # pd.DataFrame(columns=df.columns)
    # print(df_interv.shape)

    # generate intervention dag & data_timesery
    eta = 2 # 衰减系数
    # cause = UniformCause(-0.2,0.2)
    for j in range(nb_interventions):
        targets = target_list[j]
        mask_intervention.extend([targets for i in range(points_per_interv[j]*nb_time_points)])
        regimes.extend([j+1 for i in range(points_per_interv[j]*nb_time_points)])
        for k in range(points_per_interv[j]):
            g_jk = copy.deepcopy(g)
            g_jk = do_intervention(num_nodes, interv_type, intra_nodes, inter_nodes, eta, targets, g_jk)
            df_jk = gen_df_from_g(g_jk, lag, intra_nodes, inter_nodes, num_nodes, n_samples=nb_time_points, 
                                max_data_gen_trials=10000, sem_type="linear-gauss", noise_scale=1)
            df_interv = pd.concat([df_interv, df_jk], axis=0)
            print(j, k, df_interv.shape)
    # print(len(regimes), len(mask_intervention))
    _save_data(file_path, i_dataset-1, df_interv, regimes=regimes, mask=mask_intervention)  
    return g, df, intra_nodes, inter_nodes, df_interv, regimes, mask_intervention

def do_intervention(num_nodes, interv_type, intra_nodes, inter_nodes, eta, targets, g_jk):
    for tn in targets:
        if interv_type=='perfect':
            for node in intra_nodes+inter_nodes:
                g_jk.remove_edges_from([(node,f"{tn}_lag0")])
        if interv_type=='imperfect':
            for i in range(g_jk.number_of_nodes()):
                p = i//num_nodes
                org_edge = g_jk.get_edge_data(f"{i%num_nodes}_lag{p}",f"{tn}_lag0",default=0)
                if org_edge!=0: 
                    change = np.random.uniform(0.25,0.5) # (2, 5)
                    if np.random.randint(2) == 0: change *= -1
                    if org_edge['weight']>0: 
                        new_w = org_edge['weight'] + ((1/eta)**p) * change # imperfect intervention,只在原有边基础上更改强度
                    else: 
                        new_w = org_edge['weight'] - ((1/eta)**p) * change
                    g_jk.add_weighted_edges_from([(f"{i%num_nodes}_lag{p}", f"{tn}_lag0",new_w)])
    return g_jk

In [ ]:
num_datasets = 1
file_path = '/home/lipeiwen.lpw/temporal_intervention/data_timesery/imperfect/'
nb_interventions = 2
min_nb_target = 1
max_nb_target = 1
nb_all_sample = 3
num_nodes = 58
nb_time_points = 120
lag = 1
interv_type='imperfect'

for i in range(0,num_datasets):
    i_dataset = i+1
    print('i_dataset:',i_dataset)
    g, df, intra_nodes, inter_nodes, df_interv, regimes, mask_intervention = simulate_timedata(
        i_dataset,
        file_path = file_path + f'{num_nodes}_p{lag}_smp{nb_all_sample*nb_time_points}/',
        nb_interventions = nb_interventions,
        min_nb_target = min_nb_target, 
        max_nb_target = max_nb_target,
        nb_all_sample = nb_all_sample,
        num_nodes = num_nodes,
        nb_time_points = nb_time_points,
        lag = lag,
        interv_type=interv_type
        )

In [ ]:
# g, df, intra_nodes, inter_nodes = gen_stationary_dyn_net_and_df(
#         num_nodes = 58, n_samples = 120,
#         p = 1, degree_intra = 3, degree_inter = 3,
#         graph_type_intra = "erdos-renyi", graph_type_inter = "erdos-renyi",
#         # w_min_intra = 0.5, w_max_intra = 2.0,
#         # w_min_inter = 0.3, w_max_inter = 0.5,
#         w_min_intra = 0.25, w_max_intra = 1.0,
#         w_min_inter = 0.25, w_max_inter = 1.0,
#         w_decay = 2.0,
#         sem_type = "linear-gauss", noise_scale  = 1,
#         max_data_gen_trials  = 3000
#         )